## (Упрощенная) ERD сущностей в Clickup
![test](ClickupERD.png "Title")

 Команда - основная верхнеуровневая сущность. Название говорит само за себя. Нас интересует исключительно в разрезе team_id, который используется в некоторых эндпойнтах API
 В команде может быть несколько воркспейсов, в которых, в свою очередь, содержатся листы, которые могут быть сгруппированы в папки, но могут быть и прямыми детьми воркспейса.
 Лист - основная сущность, интересующая нас в контексте сборки датасетов, т.к. это коллекция тасков, которые, в свою очередь, содержат всю интересующую нас информацию (дата создания, статус и т.п.). Стоит обратить внимание, что мы можем определить кастомные поля для использования в тасках (например, причина отвала и т.п.). При запросе к API эти поля будут доступны по отдельному ключу custom_fields.
В интерфейсе все это выглядит как-то так:

![Interface](Entities.png "Ent")

In [1]:
import pandas as pd
import json
import ClickUpAPI as cua
import numpy as np
from ast import literal_eval
from math import ceil
from typing import List
from pprint import pprint
from urllib.parse import quote_plus

Первым делом, как и для практически любого API, нам понадобится токен. В Кликапе есть два вида токенов: персональный, который можно получить в секции Apps настроек своего профиля и полноценный authorization code grant OAuth2. Здесь мы будем использовать персональный токен. ![SegmentLocal](PersToken3.gif "segment")

In [2]:
token = os.environ.get("ClickUpToken")

In [4]:
# Проверим, правильно ли API возвращает юзера, от лица которого делаются запросы:
test_class = cua.ClickupClient(token)
res = test_class.get_user()
print(res)

ValueError: Какие-то беды. Код ошибки: 400

In [ ]:
# Получим айдишник команды, к которой у нас есть доступ:
team_id = test_class.get_teams(id_only=True)
print(team_id)

4514698


In [ ]:
# получим всю инфу про интересующий нас спейс (в данном случае CRM)
space = test_class.get_space_by_name(team_id, name='CRM')
print(space)

{'id': '6849848', 'name': 'CRM', 'private': False, 'statuses': [{'id': 'p6849848_hGuYhhYV', 'status': 'Open', 'type': 'open', 'orderindex': 0, 'color': '#d3d3d3'}, {'id': 'p6849848_aJ5x29At', 'status': 'concept', 'type': 'custom', 'orderindex': 1, 'color': '#FFCC00'}, {'id': 'p6849848_uHca5I69', 'status': 'in progress', 'type': 'custom', 'orderindex': 2, 'color': '#FF540D'}, {'id': 'p6849848_rQtXfgXn', 'status': 'running', 'type': 'custom', 'orderindex': 3, 'color': '#FF00DF'}, {'id': 'p6849848_Yuo3uSOo', 'status': 'review', 'type': 'custom', 'orderindex': 4, 'color': '#A875FF'}, {'id': 'p6849848_e0PVKVnw', 'status': 'Closed', 'type': 'closed', 'orderindex': 5, 'color': '#6bc950'}], 'multiple_assignees': True, 'features': {'due_dates': {'enabled': True, 'start_date': True, 'remap_due_dates': False, 'remap_closed_due_date': False}, 'sprints': {'enabled': False}, 'time_tracking': {'enabled': True, 'harvest': False, 'rollup': False}, 'points': {'enabled': False}, 'custom_items': {'enabled

In [ ]:
# получим всю инфу про интересующие нас листы (в данном случае 'Учащиеся' и 'Лиды (родители)')
studs = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Учащиеся')
pars = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Лиды (родители)')

In [ ]:
tags = test_class.get_tags(space['id'])
tags_names = [tag['name'] for tag in tags['tags']]
tags_to_keep = [quote_plus(tag) for tag in tags_names if tag not in ['тестовый', 'технический']]
tags_to_keep.append('')

In [ ]:
#заберем данные по таскам 
tasks = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[pars['id']])

page=0&list_ids[]=44610695&include_closed=true
page=1&list_ids[]=44610695&include_closed=true
page=2&list_ids[]=44610695&include_closed=true
page=3&list_ids[]=44610695&include_closed=true
page=4&list_ids[]=44610695&include_closed=true
page=5&list_ids[]=44610695&include_closed=true
page=6&list_ids[]=44610695&include_closed=true
page=7&list_ids[]=44610695&include_closed=true
page=8&list_ids[]=44610695&include_closed=true
page=9&list_ids[]=44610695&include_closed=true
page=10&list_ids[]=44610695&include_closed=true
page=11&list_ids[]=44610695&include_closed=true
page=12&list_ids[]=44610695&include_closed=true
page=13&list_ids[]=44610695&include_closed=true
page=14&list_ids[]=44610695&include_closed=true
page=15&list_ids[]=44610695&include_closed=true


In [ ]:
#поскольку get_all_tasks возвращает нам список nested джейсонов, напишем функцию, которая сложит все это добро в один датафрейм
#подробнее тут https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
def df_getter(data:List[dict], key_getter='tasks', **kwargs) -> pd.DataFrame:
    """
    transforms a list of jsons into a single dataframe using pd.json_normalize
    """
    res = list()
    for chunk in data:
        res.append(pd.json_normalize(chunk[key_getter], **kwargs))
    return pd.concat(res)

In [ ]:
#сформируем датасет лидов (то бишь родителей)
par_names = df_getter(tasks,
                      record_path='custom_fields',
                      meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                           ['status', 'status'], ['status', 'type'],
                           ['creator', 'email'], 'due_date', 'start_date', 'tags'],
                      meta_prefix='standard_')
# вытащим теги, чтобы потом выкинуть тестовиков
tags = par_names[['standard_tags', 'standard_id']]
#отберем релевантные столбцы и пивотнем кастомные поля
par_names = par_names[par_names['name'].isin(['Источник трафика', 'Канал привлечения', 'причина отказа', 'Дата заявки'])][['name', 'value', 'standard_name', 'standard_id', 'standard_text_content']]
par_names = par_names.pivot(index=['standard_name', 'standard_id', 'standard_text_content'], columns='name', values='value').reset_index()
par_names = par_names.set_index('standard_id')
#резберем строку с utm-метками и превратим ее в 3 столбца
par_names['Источник трафика'] = par_names['Источник трафика'].fillna("")
par_names['utms'] = par_names['Источник трафика'].str.findall(r"(?<==)([\w\-_\s]*)")
par_names['keys'] = par_names['Источник трафика'].str.findall(r"([\w\-_]+)(?==)")
#astype здесь используется как способ конвертнуть пустые и непустые листы в False и True соответственно
mask = par_names['keys'].astype(bool)
antimask = par_names['keys'].astype(bool) == False
utms = par_names.loc[mask, ('utms', 'keys')]
no_utms = par_names.loc[antimask, ('Источник трафика', 'utms')]
#Конструируем словарь utm-ок для тех у кого они заполнены
utms['dict'] = utms.apply(lambda x: {x['keys'][i]:x['utms'][i] for i in range(len(x['keys']))}, axis=1)
#Конструируем словарь utm-ок по умолчанию, для тех у кого они не прокинуты
no_utms['dict'] = no_utms.apply(lambda x: {"utm_source":x['Источник трафика']}, axis=1)
#Собираем это воедино, разбиваем на столбцы и джойним к родительскому датафрейму
full_utms = pd.concat([utms['dict'], no_utms['dict']])
stand_index = full_utms.index
full_utms = pd.DataFrame(full_utms.tolist())
full_utms = full_utms.set_index(stand_index)
par_names = pd.concat([par_names, full_utms], axis=1)

In [ ]:
#отдельно обработаем случай с автоматически сгенеренными лидами с fb
par_names.loc[par_names['standard_text_content'].str.contains('fb_leads', na=False), ('utm_source', 'utm_campaign')] = ('facebook_empty', 'yandex-math_test-offer_leadform')
#par_names.loc[par_names['standard_text_content'].str.contains('fb_leads', na=False), ('utm_source', 'utm_campaign')] = ('fb', 'leads')
par_names = par_names.reset_index().rename({'index': 'standard_id'}, axis=1)
par_names = par_names[['standard_name', 'standard_id', 'utm_source', 'utm_campaign', 'utm_term', 'utm_medium', 'utm_content', 'причина отказа', 'Дата заявки']]
par_names['standard_name'] = par_names['standard_name'].replace(r'\n', ' ', regex=True)
#Конвертнем дату заявки в дату
par_names['Дата заявки'] = (par_names['Дата заявки'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")).dt.date

In [ ]:
#сформируем список тестовых айдишников
tags_df= tags.explode('standard_tags')
tags_df = tags_df[~tags_df['standard_tags'].isna()]
tags_df['name'] = tags_df['standard_tags'].apply(lambda x: x.get('name'))
throwaway = tags_df.query("name == 'тестовый' or name == 'технический' ")['standard_id'].unique()

In [ ]:
#получим плоский лист всех родительских айдишников
parent_ids = np.setdiff1d(par_names['standard_id'].unique(), throwaway)
task_ids = list(parent_ids)
print(len(task_ids))
history = test_class.get_time_in_status(task_ids)

1460


In [ ]:
#поскольку get_time_in_status возвращает нам список словарей,
#в роли ключей у которых выступают родительские айдишники
#соберем это в единый словарь и превратим в датафрейм
flattened = dict()
for task_collection in history:
    flattened.update(task_collection)
flat_history = {task_id:flattened[task_id]['status_history'] for task_id in flattened}
flat_df = pd.DataFrame.from_dict(flat_history, orient='index')
#сейчас у нас столбцы со словарями для истории каждого статуса - пометим их префиксом
columns = ['col'+str(i) for i in range(len(flat_df.columns))]
flat_df.columns = columns
flat_df = flat_df.reset_index()
#разберем вложенные словари в каждой группе столбцов через json_normalize
#и дропнем столбцы со словарями
flat_df = flat_df.where(flat_df.notna(), lambda x: [{}])
for col in columns:
    flat_df = flat_df.join(pd.json_normalize(flat_df[col]).add_prefix(col+'_'))
flat_df = flat_df.drop(columns=columns)
flat_df.head()


,index,col0_status,col0_color,col0_type,col0_orderindex,col0_total_time.by_minute,col0_total_time.since,col1_status,col1_color,col1_type,...,col13_status,col13_color,col13_type,col13_total_time.by_minute,col13_total_time.since,col14_status,col14_color,col14_type,col14_total_time.by_minute,col14_total_time.since
0,119e7p5,0. новый лид,#d3d3d3,open,0,604,1625603197254,3.слот встречи назначен,#f9d900,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,119eteq,0. новый лид,#d3d3d3,open,0,566,1625606086585,2.перезвонить на неделе,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119fjhx,0. новый лид,#d3d3d3,open,0,510,1625609723280,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119g3bp,0. новый лид,#d3d3d3,open,0,425,1625615578207,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,119gcy3,0. новый лид,#d3d3d3,open,0,366,1626273866613,1. недозвон (нет контакта),#800000,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#превратим нашу таблицу из широкой в длинную и избавимся от префиксов для столбцов с одинановой сутью
#например, для time_by_minute (время в статусе)
melted_df = flat_df.fillna("").melt(id_vars=['index'])
melted_df['variable'] = melted_df['variable'].str.split("_", n=1).str[1]
#выкинем бесполезные столбцы и сделаем родительские айдишники индексами
melted_df = melted_df[~melted_df['variable'].isin(['orderindex', 'color'])]
melted_df = melted_df.set_index('index')
#соберем список отдельных датафреймов для каждого статуса и схлопнем их в одну таблицу
#NB при итерировании по групбаю он возвращает сначала ключ, а потом датафрейм по этому ключу
to_concat = [y for x,y in melted_df.groupby('variable', as_index=False)]
new_df = pd.concat(to_concat, axis=1)
#дропнем нерелевантные столбцы и переименуем оставшиеся
col_names = list(new_df.iloc[0, [i for i in range(0,len(new_df.columns),2)]])
new_df = new_df.iloc[:,[i for i in range(1,len(new_df.columns),2)]]
new_df.columns = col_names
new_df = new_df.rename(columns={'total_time.by_minute':'min_passed', 'total_time.since': 'start_dt'})
#конвертнем столбец со временем в формат даты
new_df['start_dt'] = new_df['start_dt'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")
new_df.head()

,status,min_passed,start_dt,type
index,,,,
119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open
119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open
119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open
119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open
119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open


In [ ]:
#получим список всех детей из кликапа
kids = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[studs['id']])

page=0&list_ids[]=46598432&include_closed=true
page=1&list_ids[]=46598432&include_closed=true
page=2&list_ids[]=46598432&include_closed=true


In [ ]:
#как и родителями, сложим все в один датафрейм
df = df_getter(kids,
               record_path='custom_fields',
               meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                    ['status', 'status'], ['status', 'type'],
                    ['creator', 'email'], 'due_date', 'start_date'],
               meta_prefix="standard_")
df.columns

Index(['id', 'name', 'type', 'date_created', 'hide_from_guests', 'required',
       'value', 'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id',
       'type_config.options', 'standard_id', 'standard_name',
       'standard_text_content', 'standard_date_created',
       'standard_date_updated', 'standard_date_closed',
       'standard_status.status', 'standard_status.type',
       'standard_creator.email', 'standard_due_date', 'standard_start_date'],
      dtype='object')

In [ ]:
#выкинем тестовых юзеров
df = df[df['standard_status.status']!='тестовый']
df.sample(5)

,id,name,type,date_created,hide_from_guests,required,value,type_config.fields,type_config.field_inverted_name,type_config.linked_subcategory_access,...,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
624,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,Инна Хинкиладзе,NaN,NaN,NaN,...,Катя Киселёва,\n,1618246840568,1619503352368,1619503352368,отвал (скрыть ученика),closed,rahmatulina@yandex-team.ru,None,None
466,21542971-21af-4389-aaae-e5a56e4c5fde,купленные пакеты,list_relationship,1615539828118,False,False,"[{'id': 'ht689r', 'name': 'Anna Denisova', 'st...",[{'field': 'cf_aef0fa70-5268-43e6-9fab-5b91a05...,Ребенок в CRM,True,...,Степан Денисов,\n,1619427231436,1627670987511,None,"пакет закончился, звоним",custom,k-perfileva@yandex-team.ru,1628211600000,None
644,b261ecb0-58b9-4a4e-bcae-1e1cafb9562f,Цель ученика и сроки достижения цели,short_text,1611823651718,False,False,NaN,NaN,NaN,NaN,...,Екатерина Сазонова,\n,1626539809434,1626539811274,None,преобразовать,open,s-dayana21@yandex-team.ru,None,None
459,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'id': 'hh445e', 'name': 'Анна Денисова', 'st...",[{'field': 'cf_b261ecb0-58b9-4a4e-bcae-1e1cafb...,Учащийся,True,...,Степан Денисов,\n,1619427231436,1627670987511,None,"пакет закончился, звоним",custom,k-perfileva@yandex-team.ru,1628211600000,None
1100,b261ecb0-58b9-4a4e-bcae-1e1cafb9562f,Цель ученика и сроки достижения цели,short_text,1611823651718,False,False,NaN,NaN,NaN,NaN,...,Катя Перемитина,\n,1624528148137,1627671706874,None,"пакет закончился, звоним",custom,rahmatulina@yandex-team.ru,1628211600000,None


In [ ]:
#удалим не нужные нам столбцы
to_drop = ['type_config.default', 'type_config.placeholder', 'type_config.new_drop_down', 'type_config.options',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id']
df = df.drop(columns=df.columns.intersection(to_drop))
df.head()

,id,name,type,date_created,hide_from_guests,required,value,standard_id,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
0,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,NaN,bv2u1r,Виталий Карпушкин,\n,1628177878379,1628177910964,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
1,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,bv2u1r,Виталий Карпушкин,\n,1628177878379,1628177910964,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
2,4a4363cf-e27e-4b59-b781-63cb161a43e3,Расписание,short_text,1613142022298,False,False,NaN,bv2u1r,Виталий Карпушкин,\n,1628177878379,1628177910964,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
3,a7d9f126-7ed3-48ea-8a1d-adcf5b1733fb,Родитель в воронке лидов,list_relationship,1613761441536,False,False,"[{'id': '1afy7pv', 'name': 'Нэлли', 'status': ...",bv2u1r,Виталий Карпушкин,\n,1628177878379,1628177910964,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None
4,e6cb4ac4-e138-4895-968d-9ebccaa12c78,Согласие на видеозапись,labels,1613132651997,False,False,NaN,bv2u1r,Виталий Карпушкин,\n,1628177878379,1628177910964,None,первая покупка,custom,s-dayana21@yandex-team.ru,None,None


In [ ]:
#пивотнем столбец name (каждое уникальное значение из этого столбца становится отдельным столбцом в пивотизированном датафрейме)
#поскольку столбец 'родитель в воронке лидов' в кликапе является ссылкой на другой лист, API возвращает нам этот лист, а пандас
#читает его как питоновский лист. В нашем случае в нем всегда один элемент, поэтому explode не увеличивает кол-во строк в датафрейме
#а просто избавляет нас от листа в cтолбце, превращая ее в столбец словарей
df2 = df.copy()
df2 = df2.pivot(index='standard_id', columns='name', values='value')
df2 = df2.explode('Родитель в воронке лидов')
df2.head()

name,Shared with me,Инциденты,Преподаватель списка учащиеся,Прогулы,Пройденные занятия пакета (старое),Расписание,Родитель в воронке лидов,Согласие на видеозапись,Цель ученика и сроки достижения цели,класс,купленные пакеты,первое занятие
standard_id,,,,,,,,,,,,
6fynn4,[],NaN,Дарья Карпухина,NaN,11\02 перерыв с 22\02 до 28\02,"чт 18-00 - 60 мин, пн 20-00 - 15 мин","{'id': '69yhrv', 'name': 'Татьяна Рушечникова ...",[1171db5a-754c-4178-85f1-67847ab15f03],"Дочь 6 класс, не очень любит математику, после...",NaN,"[{'id': 'ejzw6t', 'name': 'Варя Рушечникова па...",1613005200000
6fynn5,NaN,NaN,"Светлана Зайцева (15 мин), Сергей Христолюбов ...",NaN,16\02,"вт 16-00 60 мин, чт 16-00 15 мин","{'id': '6fu8tt', 'name': 'Анна Гуляева', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6w', 'name': 'Иван Гуляев пакет 4...",1613437200000
6fyppm,NaN,NaN,"Светлана Зайцева (15 мин), Анна И (60 мин)",NaN,NaN,"вскр 14-00 - 60 мин, вт 16-00 - 15 мин","{'id': 'd6zdz6', 'name': 'Ольга', 'status': '9...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'fk0rm6', 'name': 'Даша пакет 4+4 купл...",1612054800000
6hqz5e,[],NaN,Антон Загривин,NaN,15\02,"пн 12-00 60 мин, чт 12-00 15 мин","{'id': '6dy7aj', 'name': 'Марина Таран', 'stat...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6r', 'name': 'Аня пакет 4+4 купле...",1613350800000
6huakn,NaN,NaN,Света и Никита,NaN,27\02,"среда 17-00 60 мин, пятница 18-30 15 мин","{'id': '6fut3z', 'name': 'Наталья Кузнецова', ...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,"[{'id': 'ejzw6c', 'name': 'Вася Кузнецов пакет...",1613523600000


In [ ]:
#читаем наш столбец со словарями в отдельный датафрейм, добавляем префикс к именам столбцов
#чтобы избежать конфликтов при последующем мердже
parent = df2['Родитель в воронке лидов'].apply(pd.Series).add_prefix("par_")
parent = parent[['par_id', 'par_name']]
parent.head()

,par_id,par_name
standard_id,,
6fynn4,69yhrv,Татьяна Рушечникова (Болгова)
6fynn5,6fu8tt,Анна Гуляева
6fyppm,d6zdz6,Ольга
6hqz5e,6dy7aj,Марина Таран
6huakn,6fut3z,Наталья Кузнецова


In [ ]:
#проделываем то же самое для столбца со ссылкой на купленные пакеты
packages = df2['купленные пакеты'].explode().apply(pd.Series).add_prefix("purch_")
packages = packages[['purch_id']]
packages.head()

,purch_id
standard_id,
6fynn4,ejzw6t
6fynn4,71t1d4
6fynn4,7dyck9
6fynn4,gz8w39
6fynn4,gk7nub


In [ ]:
#получаем инфу о списке покупок в кликапе
purch_list = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Заявки на покупку')
print(purch_list)

{'id': '48853634', 'name': 'Заявки на покупку', 'orderindex': 0, 'status': None, 'priority': None, 'assignee': None, 'task_count': 267, 'due_date': None, 'start_date': None, 'folder': {'id': '23627065', 'name': 'hidden', 'hidden': True, 'access': True}, 'space': {'id': '6849848', 'name': 'CRM', 'access': True}, 'archived': False, 'override_statuses': True, 'permission_level': 'create'}


In [ ]:
#забираем все таски из этого списка
purchases = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids = [purch_list['id']])

page=0&list_ids[]=48853634&include_closed=true
page=1&list_ids[]=48853634&include_closed=true
page=2&list_ids[]=48853634&include_closed=true
page=3&list_ids[]=48853634&include_closed=true


In [ ]:
#и внвь соберем эту инфу в плоский датафрейм, выкинем тесовых и оставим только релевантные столбцы
purch_df = df_getter(purchases, record_path='custom_fields', meta=['id', ['status', 'status']],meta_prefix='standard_')
purch_df = purch_df[(purch_df['name']=='Ссылка на ребенка в админке') & (purch_df['standard_status.status']!='тестовый')][['value', 'standard_id']].dropna()
#вытащим praktikum_id из юрла
purch_df['praktikum_id'] = purch_df['value'].str.split("/").str[-1]
purch_df.head()

,value,standard_id,praktikum_id
3,https://praktikum-admin.yandex-team.ru/math/st...,rga4d1,11894532
13,https://praktikum-admin.yandex-team.ru/math/st...,rg926v,11891093
23,https://praktikum-admin.yandex-team.ru/math/st...,rg7r77,11887154
33,https://praktikum-admin.yandex-team.ru/math/st...,rapu0q,11872216
43,https://praktikum-admin.yandex-team.ru/math/st...,rap6vv,11870727


In [ ]:
#сопоставим айдишники детей с их айдишниками в практикуме
kids_links = packages.reset_index().merge(purch_df, how='left', left_on='purch_id', right_on='standard_id')
kids_links = kids_links[['purch_id', 'praktikum_id', 'standard_id_x']].groupby('standard_id_x')['praktikum_id'].last()
kids_links.head()

standard_id_x
6fynn4    7210563
6fynn5    7203187
6fyppm       None
6hqz5e    7245837
6huakn       None
Name: praktikum_id, dtype: object

In [ ]:
#теперь сджойним детей с их родителями
full_lineage = parent.merge(kids_links, how='left', left_index=True, right_index=True)
full_lineage.sample(5)

,par_id,par_name,praktikum_id
standard_id,,,
p7gq23,15xwwme,Наталья Губарева,11346440
j79kv3,85uk8b,Надежда Логунова,7864393
pzdmu3,p39hw5,Олеся Царакхова,11732130
ptbu2v,161wmxd,Иван Журавлев (Yandex-math.ru),11634925
mkdww6,840hyc,Юлия Кочанова,9814915


In [ ]:
#наконец, смерджим все это воедино с датафреймом родительских атрибутов
fin_df = new_df.reset_index().merge(full_lineage, how='left', left_on='index', right_on='par_id')
fin_df = fin_df[fin_df['type']!=""]
fin_df = fin_df.merge(par_names, how='left', left_on='index', right_on='standard_id')
fin_df.sample(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_term,utm_medium,utm_content,причина отказа,Дата заявки
2554,raq9y4,3.слот встречи назначен,840,2021-08-05 09:18:54.436,custom,NaN,NaN,NaN,Марина Бобылева,raq9y4,fb,math_fb_leadgen_september_offer,NaN,cpc,offer3_zapishis_v_september,NaN,2021-08-04
3398,6dqrmy,ждет связи с оператором,501,2021-02-05 10:18:23.933,custom,NaN,NaN,NaN,Оксана,6dqrmy,facebook_leads,NaN,NaN,NaN,NaN,1,2021-02-05
2673,ybe7hf,3.слот встречи назначен,2660,2021-06-17 13:28:07.880,custom,ybe7hf,Татьяна Павлова (Валентин Артемьев),10005004,Татьяна Павлова (Валентин Артемьев),ybe7hf,leto landing,NaN,NaN,NaN,NaN,NaN,2021-06-16
5087,zkmraj,1. недозвон (пингуем нед.),11026,2021-07-05 19:15:38.628,custom,NaN,NaN,NaN,Раисат Басраева,zkmraj,,NaN,NaN,NaN,NaN,0,2021-07-03
96,139kujg,0. новый лид,11,2021-07-31 19:46:04.956,open,NaN,NaN,NaN,Карина,139kujg,leto landing,NaN,NaN,NaN,NaN,NaN,2021-07-11
664,gz76ew,3.слот встречи назначен,2431,2021-04-10 17:45:08.358,custom,gz76ew,Даниил Киселёв,7431678,Даниил Киселёв,gz76ew,facebook,NaN,NaN,NaN,NaN,7,2021-04-10
3288,18dxxgm,1. недозвон 2 сутки,1277,2021-07-28 12:23:29.317,custom,NaN,NaN,NaN,Анастасия Коркина,18dxxgm,,NaN,NaN,NaN,NaN,2,2021-07-27
6090,k98rbn,"2.перезвонить, узнать слот",78,2021-05-23 17:27:31.410,custom,NaN,NaN,NaN,Ирина Скачко,k98rbn,other2,NaN,NaN,NaN,NaN,1,2021-05-22
746,ht5u74,1. недозвон 1 сутки,434,2021-04-26 11:16:10.617,custom,NaN,NaN,NaN,Диляра,ht5u74,other2,NaN,NaN,NaN,NaN,NaN,2021-04-26
4614,hf58w6,1. недозвон 2 раза,1366,2021-04-21 12:16:32.355,custom,NaN,NaN,NaN,Самира,hf58w6,google,,,NaN,NaN,0,2021-04-19


In [ ]:
#добавим мэппинг статусов из файла
statuses = pd.read_excel('mappings.xlsx', engine='openpyxl', sheet_name='statuses', index_col=1)
fin_df = fin_df.merge(statuses, left_on='status', right_index=True, how='left')
fin_df.head(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,...,utm_campaign,utm_term,utm_medium,utm_content,причина отказа,Дата заявки,этап,Созвон,Бронирование,Встреча
0,119e7p5,0. новый лид,604,2021-07-06 23:26:37.254,open,119e7p5,Светлана (Иван),10994925,Светлана (Иван),119e7p5,...,NaN,NaN,NaN,NaN,NaN,2021-07-06,0 заявка,0.0,0.0,0.0
1,119eteq,0. новый лид,566,2021-07-07 00:14:46.585,open,NaN,NaN,NaN,Александра Борисова,119eteq,...,NaN,NaN,NaN,NaN,2,2021-07-07,0 заявка,0.0,0.0,0.0
2,119fjhx,0. новый лид,510,2021-07-07 01:15:23.280,open,NaN,NaN,NaN,КАТЕРИНА,119fjhx,...,NaN,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
3,119g3bp,0. новый лид,425,2021-07-07 02:52:58.207,open,119g3bp,Анна(Денис),11200929,Анна(Денис),119g3bp,...,NaN,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
4,119gcy3,0. новый лид,366,2021-07-14 17:44:26.613,open,NaN,NaN,NaN,Марина Белькова,119gcy3,...,NaN,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
5,119gmeg,0. новый лид,303,2021-07-07 05:00:28.312,open,NaN,NaN,NaN,Светлана Керебко,119gmeg,...,NaN,NaN,NaN,NaN,3,2021-07-07,0 заявка,0.0,0.0,0.0
6,119gr05,0. новый лид,277,2021-07-07 05:28:11.314,open,NaN,NaN,NaN,Светлана Реутова,119gr05,...,NaN,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
7,119h0p7,0. новый лид,174,2021-07-07 07:20:50.794,open,119h0p7,Галина Пеншина,11102355,Галина Пеншина,119h0p7,...,NaN,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0
8,119h459,0. новый лид,139,2021-07-07 07:52:26.718,open,NaN,NaN,NaN,светлана иванова 888,119h459,...,NaN,NaN,NaN,NaN,0,2021-07-07,0 заявка,0.0,0.0,0.0
9,119h57w,0. новый лид,128,2021-07-20 19:34:56.653,open,NaN,NaN,NaN,Светлана Андреевская,119h57w,...,NaN,NaN,NaN,NaN,NaN,2021-07-07,0 заявка,0.0,0.0,0.0


In [ ]:
fin_df['call'] = fin_df['start_dt'].where(fin_df['Созвон']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['meeting'] = fin_df['start_dt'].where(fin_df['Встреча']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['booking'] = fin_df['start_dt'].where(fin_df['Бронирование']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df = fin_df.drop(columns=['Созвон', 'Встреча', 'par_name', 'index', 'par_id']).rename(columns={'причина отказа': 'churn_reason'})
fin_df['lead_date'] = fin_df.groupby('standard_id')['start_dt'].transform('min')
fin_df['lead_date'] = fin_df['Дата заявки'].combine_first(fin_df['lead_date'])
fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date'] = fin_df[fin_df['Дата заявки'].isna()]['lead_date'].apply(pd.to_datetime).dt.date
fin_df = fin_df.drop(columns=['Дата заявки', 'Бронирование'])
fin_df = fin_df.rename(columns={'этап': 'etap'})
fin_df.sample(7)

,status,min_passed,start_dt,type,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_term,utm_medium,utm_content,churn_reason,etap,call,meeting,booking,lead_date
3191,2. отказ после контакта,19420,2021-07-23 11:37:40.801,custom,NaN,Алла Ихнева,185qvhg,fb_leads,NaN,NaN,NaN,NaN,1,2 выясняем цели,2021-07-22 09:31:41.725,NaT,NaT,2021-07-22
1717,2. отказ после контакта,26633,2021-07-18 11:24:58.913,custom,NaN,НАТАЛИ,15znz35,fb_leads,NaN,NaN,NaN,NaN,3,2 выясняем цели,2021-07-18 11:24:58.913,NaT,NaT,2021-07-18
3201,2.перезвонить на неделе,15412,2021-07-23 17:44:13.347,custom,NaN,Елена,185vngh,fb_leads,NaN,NaN,NaN,NaN,0,2 выясняем цели,2021-07-23 17:44:13.347,NaT,NaT,2021-07-22
5260,"8. есть оплата, нет записи",11652,2021-07-20 09:37:05.500,custom,11409859,Татьяна Павликова,15zq0kj,fb_leads,NaN,NaN,NaN,NaN,NaN,6 онбординг,2021-07-19 09:22:20.479,2021-07-19 15:49:15.203,2021-07-19 09:22:20.479,2021-07-18
4514,4.ждет встречи с куратором,12752,2021-02-10 17:50:08.189,custom,NaN,Ольга Садикова,6fut5g,facebook_leads,NaN,NaN,NaN,NaN,2,na,2021-02-10 15:44:08.753,2021-02-26 13:20:52.558,2021-06-16 18:38:05.650,2021-02-09
2113,ждет первого контакта,111,2021-02-08 12:35:21.409,open,NaN,Диана,dx49p8,facebook_leads,NaN,NaN,NaN,NaN,0,na,NaT,NaT,NaT,2021-02-04
962,0. новый лид,19,2021-07-27 18:09:42.259,open,NaN,Наталья,pth512,fb,math_fb_leadgen_september_offer,NaN,cpc,offer3_zapishis_v_september,NaN,0 заявка,NaT,NaT,NaT,2021-07-27


In [ ]:
fin_df['standard_id'].nunique()

1460

In [ ]:
fin_df.to_csv('fin1_df.csv')

# ЗЭ ЭНД